In [1]:
import pandas as pd
import numpy as np

bs24 = pd.read_csv('../data/raw/2024_batting_stats.csv')
bs25 = pd.read_csv('../data/raw/2025_batting_stats.csv')

bs24 = bs24[bs24['PA'] > 0]
bs25 = bs25[bs25['PA'] > 0]

In [2]:
def check_data(df):
    print(f"Shape: {df.shape}")

    # Check for duplicate IDs/rows, missing values, and zero-variance
    print(f"Unique IDs? {df['IDfg'].is_unique}")
    print(f"Duplicates? {df.duplicated().any()}")
    print(f"Missing values? {df.isna().any().any()}")
    print(f"Zero-variance? {(df.nunique(dropna=False) <= 1).any()}")

    # Print details if needed    
    if not df['IDfg'].is_unique:
        print("Duplicate IDs:")
        dup_mask = df['IDfg'].duplicated(keep=False)
        dup_groups = df[dup_mask].groupby('IDfg')
        for idfg, group in dup_groups:
            print(f"  - IDfg {idfg}: indices {list(group.index)}")
            
    if df.duplicated().any():
        print("Duplicate rows:")
        dup_mask = df.duplicated(keep=False)
        dup_groups = df[dup_mask].groupby(list(df.columns), dropna=False)
        for _, group in dup_groups:
            print(f"  - {list(group.index)}")
            
    if df.isna().any().any():
        print("Columns with missing values:")
        na_pct = df.isna().mean()
        na_cols = na_pct[na_pct > 0].sort_values(ascending=False)
        for col, pct in na_cols.items():
            print(f"  - {col}: {pct:.1%}")
            
    if (df.nunique(dropna=False) <= 1).any():
        print("Columns with zero variance:")
        zero_var_cols = [col for col in df.columns if df[col].nunique(dropna=False) <= 1]
        for col in zero_var_cols:
            print(f"  - {col}")
    
    df.info()

In [3]:
print('Season: 2024')
check_data(bs24)

Season: 2024
Shape: (651, 320)
Unique IDs? True
Duplicates? False
Missing values? True
Zero-variance? True
Columns with missing values:
  - FT% (sc): 100.0%
  - UN% (sc): 100.0%
  - vFT (sc): 100.0%
  - FT-X (sc): 100.0%
  - FT-Z (sc): 100.0%
  - wFT (sc): 100.0%
  - wFT/C (sc): 100.0%
  - PO%: 100.0%
  - FO-Z (sc): 91.2%
  - vFO (sc): 91.2%
  - FO-X (sc): 91.2%
  - wFO (sc): 91.2%
  - wFO/C (sc): 91.2%
  - FO% (sc): 91.2%
  - wXX (pi): 91.1%
  - XX-X (pi): 91.1%
  - XX-Z (pi): 91.1%
  - wXX/C (pi): 91.1%
  - XX% (pi): 91.1%
  - vXX (pi): 91.1%
  - wSB/C (pi): 89.2%
  - SB-X (pi): 89.2%
  - vSB (pi): 89.2%
  - wSB (pi): 89.2%
  - SB% (pi): 89.2%
  - SB-Z (pi): 89.2%
  - wCS (pi): 88.8%
  - vCS (pi): 88.8%
  - CS-X (pi): 88.8%
  - CS% (pi): 88.8%
  - wCS/C (pi): 88.8%
  - CS-Z (pi): 88.8%
  - vSC (sc): 87.7%
  - wSC (sc): 87.7%
  - SC-Z (sc): 87.7%
  - SC-X (sc): 87.7%
  - wSC/C (sc): 87.7%
  - SC% (sc): 87.7%
  - FRM: 84.8%
  - EP% (sc): 73.3%
  - wEP (sc): 73.3%
  - EP-Z (sc): 73.3%
 

In [4]:
print('Season: 2025')
check_data(bs25)

Season: 2025
Shape: (673, 320)
Unique IDs? True
Duplicates? False
Missing values? True
Zero-variance? True
Columns with missing values:
  - PO%: 100.0%
  - FT-X (sc): 100.0%
  - SB% (pi): 100.0%
  - FT-Z (sc): 100.0%
  - UBR: 100.0%
  - wSB/C (pi): 100.0%
  - wFT/C (sc): 100.0%
  - UN% (sc): 100.0%
  - FT% (sc): 100.0%
  - SB-Z (pi): 100.0%
  - vSB (pi): 100.0%
  - wGDP: 100.0%
  - SB-X (pi): 100.0%
  - wFT (sc): 100.0%
  - wSB (pi): 100.0%
  - vFT (sc): 100.0%
  - vSC (sc): 99.0%
  - wSC/C (sc): 99.0%
  - SC% (sc): 99.0%
  - SC-X (sc): 99.0%
  - wSC (sc): 99.0%
  - SC-Z (sc): 99.0%
  - CS% (pi): 97.9%
  - vCS (pi): 97.9%
  - CS-Z (pi): 97.9%
  - wCS (pi): 97.9%
  - CS-X (pi): 97.9%
  - wCS/C (pi): 97.9%
  - KNv: 97.2%
  - wKN: 97.2%
  - KN%: 97.2%
  - wKN/C: 97.2%
  - wKN (pi): 95.4%
  - KN-X (pi): 95.4%
  - wKN/C (pi): 95.4%
  - KN% (pi): 95.4%
  - vKN (pi): 95.4%
  - KN-Z (pi): 95.4%
  - KN-X (sc): 94.1%
  - wKN/C (sc): 94.1%
  - wKN (sc): 94.1%
  - KN-Z (sc): 94.1%
  - vKN (sc): 94

In [5]:
set(bs24.columns) == set(bs25.columns)

True

In [6]:
df = bs25.copy()

# Filtered columns
pitch_splits = ['Pitches', 'Balls', 'Strikes', 'FB% (Pitch)', 'SL%', 'CT%', 'CB%', 'CH%', 'SF%', 'KN%', 'XX%', 'PO%', 'wFB/C', 'wSL/C', 
                'wCT/C', 'wCB/C', 'wCH/C', 'wSF/C', 'wKN/C', 'FBv', 'SLv',	'CTv', 'CBv', 'CHv', 'SFv',	'KNv']
df = df.drop(columns=pitch_splits)

value = ['Bat', 'RAR', 'WAR', 'Dol', 'WPA', '-WPA', '+WPA', 'WPA/LI', 'RE24', 'REW', 'L-WAR']
df = df.drop(columns=value)

statcast = [c for c in df.columns if '(sc)' in c]
df = df.drop(columns=statcast)

pitch_info = [c for c in df.columns if '(pi)' in c]
df = df.drop(columns=pitch_info)

plus = [c for c in df.columns if c[-1] == '+']
plus.append('LD+%')
df = df.drop(columns=plus)

rate = [c for c in df.columns if ('%' in c) or ('/' in c)]
rate.extend(['AVG', 'Pace'])
df = df.drop(columns=rate)

weighted = [c for c in df.columns if c[0] == 'w']
df = df.drop(columns=weighted)

expected = [c for c in df.columns if c[0] == 'x']
df = df.drop(columns=expected)

# Other columns
metadata = ['IDfg', 'Season', 'Name', 'Team', 'Age', 'Age Rng']
count = ['G', 'AB', 'PA', 'H', '1B', '2B',	'3B', 'HR',	'R', 'RBI',	'BB', 'IBB', 'SO', 'HBP', 'SF',	'SH', 'GDP', 'GB', 'FB', 
         'LD', 'IFFB', 'Pitches', 'Balls', 'Strikes', 'IFH', 'BU', 'BUH']
advanced = ['OBP', 'SLG', 'OPS', 'ISO',	'BABIP']
hitting = ['EV', 'LA', 'Barrels', 'maxEV', 'HardHit']
pinch_hitting = ['pLI', 'phLI', 'PH']
base_running = ['SB', 'CS', 'Spd', 'UBR', 'BsR']
defense = ['Fld', 'FRM', 'Def']
misc = ['Rep', 'Clutch', 'Rep', 'Off', 'Pos', 'Lg', 'Events']

# Print categories and columns
print("Metadata:")
for c in metadata:
    print(f" - {c}")   
print("Counting metrics:")
for c in count:
    print(f" - {c}")
print("Rate metrics:")
for c in rate:
    print(f" - {c}")
print("Weighted metrics:")
for c in weighted:
    print(f" - {c}")
print("Expected metrics:")
for c in expected:
    print(f" - {c}")
print("*+ metrics:")
for c in plus:
    print(f" - {c}")
print("Statcast metrics:")
for c in statcast:
    print(f" - {c}")
print("Pitch Info metrics:")
for c in pitch_info:
    print(f" - {c}")
print("Advanced metrics:")
for c in advanced:
    print(f" - {c}")
print("Value metrics:")
for c in value:
    print(f" - {c}")
print("Hitting metrics:")
for c in hitting:
    print(f" - {c}")
print("Pinch hitting metrics:")
for c in pinch_hitting:
    print(f" - {c}")
print("Base running metrics:")
for c in base_running:
    print(f" - {c}")
print("Pitch splits:")
for c in pitch_splits:
    print(f" - {c}")
print("Defensive metrics:")
for c in defense:
    print(f" - {c}")
print("Miscellaneous metrics:")
for c in misc:
    print(f" - {c}")

Metadata:
 - IDfg
 - Season
 - Name
 - Team
 - Age
 - Age Rng
Counting metrics:
 - G
 - AB
 - PA
 - H
 - 1B
 - 2B
 - 3B
 - HR
 - R
 - RBI
 - BB
 - IBB
 - SO
 - HBP
 - SF
 - SH
 - GDP
 - GB
 - FB
 - LD
 - IFFB
 - Pitches
 - Balls
 - Strikes
 - IFH
 - BU
 - BUH
Rate metrics:
 - BB%
 - K%
 - BB/K
 - GB/FB
 - LD%
 - GB%
 - FB%
 - IFFB%
 - HR/FB
 - IFH%
 - BUH%
 - O-Swing%
 - Z-Swing%
 - Swing%
 - O-Contact%
 - Z-Contact%
 - Contact%
 - Zone%
 - F-Strike%
 - SwStr%
 - Pull%
 - Cent%
 - Oppo%
 - Soft%
 - Med%
 - Hard%
 - TTO%
 - Barrel%
 - HardHit%
 - CStr%
 - CSW%
 - AVG
 - Pace
Weighted metrics:
 - wOBA
 - wRAA
 - wRC
 - wFB
 - wSL
 - wCT
 - wCB
 - wCH
 - wSF
 - wKN
 - wSB
 - wGDP
Expected metrics:
 - xBA
 - xSLG
 - xwOBA
*+ metrics:
 - wRC+
 - AVG+
 - BB%+
 - K%+
 - OBP+
 - SLG+
 - ISO+
 - BABIP+
 - GB%+
 - FB%+
 - HR/FB%+
 - Pull%+
 - Cent%+
 - Oppo%+
 - Soft%+
 - Med%+
 - Hard%+
 - LD+%
Statcast metrics:
 - FA% (sc)
 - FT% (sc)
 - FC% (sc)
 - FS% (sc)
 - FO% (sc)
 - SI% (sc)
 - SL% (sc)

In [7]:
# Drop columns
df = bs25.copy()

drop = []
drop.extend(expected)
drop.extend(statcast)
drop.extend(pitch_info)
drop.extend(base_running)
drop.extend(defense)
drop.extend(misc)
drop.extend(value)
drop.extend(c for c in weighted if c != 'wOBA')
drop.extend(plus)
drop.extend(pitch_splits)
drop.extend(pinch_hitting)
drop.extend(['Age Rng', 'G', 'R', 'RBI', 'SH', 'IBB', 'TTO%', 'Pace', 'CStr%'])

for c in drop:
    print(f" - {c}")

df = df.drop(columns=drop)
df.shape
# df.to_csv('../data/processed/2025_batting_stats_cleaned.csv', index=False)

 - xBA
 - xSLG
 - xwOBA
 - FA% (sc)
 - FT% (sc)
 - FC% (sc)
 - FS% (sc)
 - FO% (sc)
 - SI% (sc)
 - SL% (sc)
 - CU% (sc)
 - KC% (sc)
 - EP% (sc)
 - CH% (sc)
 - SC% (sc)
 - KN% (sc)
 - UN% (sc)
 - vFA (sc)
 - vFT (sc)
 - vFC (sc)
 - vFS (sc)
 - vFO (sc)
 - vSI (sc)
 - vSL (sc)
 - vCU (sc)
 - vKC (sc)
 - vEP (sc)
 - vCH (sc)
 - vSC (sc)
 - vKN (sc)
 - FA-X (sc)
 - FT-X (sc)
 - FC-X (sc)
 - FS-X (sc)
 - FO-X (sc)
 - SI-X (sc)
 - SL-X (sc)
 - CU-X (sc)
 - KC-X (sc)
 - EP-X (sc)
 - CH-X (sc)
 - SC-X (sc)
 - KN-X (sc)
 - FA-Z (sc)
 - FT-Z (sc)
 - FC-Z (sc)
 - FS-Z (sc)
 - FO-Z (sc)
 - SI-Z (sc)
 - SL-Z (sc)
 - CU-Z (sc)
 - KC-Z (sc)
 - EP-Z (sc)
 - CH-Z (sc)
 - SC-Z (sc)
 - KN-Z (sc)
 - wFA (sc)
 - wFT (sc)
 - wFC (sc)
 - wFS (sc)
 - wFO (sc)
 - wSI (sc)
 - wSL (sc)
 - wCU (sc)
 - wKC (sc)
 - wEP (sc)
 - wCH (sc)
 - wSC (sc)
 - wKN (sc)
 - wFA/C (sc)
 - wFT/C (sc)
 - wFC/C (sc)
 - wFS/C (sc)
 - wFO/C (sc)
 - wSI/C (sc)
 - wSL/C (sc)
 - wCU/C (sc)
 - wKC/C (sc)
 - wEP/C (sc)
 - wCH/C (sc)
 - w

(673, 65)

In [8]:
df = pd.read_csv('../data/processed/2025_batting_stats_cleaned.csv')
df.shape

(1470, 65)